### This Notebook will contain code for creating a reduced size map file.

In [1]:
# Import Required Libraries

import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Import ipywidgets and display an interactive integer slider widget

import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

In [3]:
# Assigning Path

path = r'/Users/elia/Desktop/New_York_City_Bike'

In [4]:
# Define specific data types to enforce when loading these columns

dtype_spec = {
    "start_station_id": "string",
    "end_station_id": "string",}

In [5]:
# Import CitiBike DataSet as a DataFrame

df = pd.read_csv(os.path.join(path, 'CB_Data', 'Prepared Data', 'ny_citybike_V2.csv'), dtype=dtype_spec)

In [10]:
# Check the first 5 rows and the columns

df.head(5)

,Unnamed: 0,ride_id,bicycle_type,rent_start_time,rent_end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_latitude,start_longitude,end_latitude,end_longitude,customer_type,trip_duration,date,avg_temp,trip_count
0,0,9D0DC440CB40CF8E,electric_bike,2022-08-27 13:56:47.728,2022-08-27 14:02:56.651,Flatbush Ave & Ocean Ave,3704.04,3 St & Prospect Park West,3865.05,40.663657,-73.963014,40.668132,-73.973638,casual,6,2022-08-27,27.8,1
1,1,2214991DFBE5C4D7,electric_bike,2022-08-20 10:37:02.756,2022-08-20 10:45:56.631,Forsyth St\t& Grand St,5382.07,E 11 St & 1 Ave,5746.14,40.717798,-73.993161,40.729538,-73.984267,casual,9,2022-08-20,27.9,1
2,2,20C5D469563B6337,classic_bike,2022-08-31 18:55:03.051,2022-08-31 19:03:37.344,Perry St & Bleecker St,5922.07,Grand St & Greene St,5500.02,40.735354,-74.004831,40.721700,-74.002381,member,9,2022-08-31,25.6,1
3,3,3E8791885BC189D1,classic_bike,2022-08-02 08:05:00.250,2022-08-02 08:16:52.063,FDR Drive & E 35 St,6230.04,Grand Army Plaza & Central Park S,6839.1,40.744219,-73.971212,40.764397,-73.973715,member,12,2022-08-02,26.4,1
4,4,8DBCBF98885106CB,electric_bike,2022-08-25 15:44:48.386,2022-08-25 15:55:39.691,E 40 St & 5 Ave,6474.11,Ave A & E 14 St,5779.11,40.752052,-73.982115,40.730311,-73.980472,member,11,2022-08-25,28.1,1


In [11]:
# Create a value column and group by start and end station
# Add a helper column and count trips for each start–end station pair using a groupby operation

df['value'] = 1
df_group = df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()

In [12]:
# Display the grouped dataframe showing trip counts by station pair

df_group

,start_station_name,end_station_name,value
0,1 Ave & E 110 St,1 Ave & E 110 St,791
1,1 Ave & E 110 St,1 Ave & E 18 St,2
2,1 Ave & E 110 St,1 Ave & E 30 St,4
3,1 Ave & E 110 St,1 Ave & E 39 St,1
4,1 Ave & E 110 St,1 Ave & E 44 St,12
...,...,...,...
1011469,Yankee Ferry Terminal,Water St & Main St,4
1011470,Yankee Ferry Terminal,West St & Chambers St,6
1011471,Yankee Ferry Terminal,West St & Liberty St,4
1011472,Yankee Ferry Terminal,West Thames St,1


In [13]:
# Print the total trip count from the grouped data and the shape of the original dataframe

print(df_group['value'].sum())
print(df.shape)

29707810
(29707810, 19)


In [14]:
# Display summary statistics for the trip count values in the grouped dataframe

df_group['value'].describe()

count    1.011474e+06
mean     2.937081e+01
std      9.913254e+01
min      1.000000e+00
25%      1.000000e+00
50%      4.000000e+00
75%      1.700000e+01
max      1.199900e+04
Name: value, dtype: float64

In [15]:
# Check first few rows of the dataframe

df_group.head(5)

,start_station_name,end_station_name,value
0,1 Ave & E 110 St,1 Ave & E 110 St,791
1,1 Ave & E 110 St,1 Ave & E 18 St,2
2,1 Ave & E 110 St,1 Ave & E 30 St,4
3,1 Ave & E 110 St,1 Ave & E 39 St,1
4,1 Ave & E 110 St,1 Ave & E 44 St,12


In [16]:
# Rename the trip_count column to total_trips for clarity

df_group.rename(columns={'value': 'total_trips'}, inplace=True)

In [17]:
# Check first few rows of the dataframe

df_group.head(5)

,start_station_name,end_station_name,total_trips
0,1 Ave & E 110 St,1 Ave & E 110 St,791
1,1 Ave & E 110 St,1 Ave & E 18 St,2
2,1 Ave & E 110 St,1 Ave & E 30 St,4
3,1 Ave & E 110 St,1 Ave & E 39 St,1
4,1 Ave & E 110 St,1 Ave & E 44 St,12


In [18]:
# Count how many times each trip count value appears, sorted by index

df_group['total_trips'].sort_index().value_counts()

total_trips
1       267511
2       128039
3        77930
4        54712
5        40748
         ...  
1579         1
1375         1
4249         1
2185         1
5679         1
Name: count, Length: 1821, dtype: int64

In [19]:
# Export Cleaned DataFrame to Csv

df_group.to_csv(os.path.join(path, 'CB_Data', 'Prepared Data', 'df_groupby_final.csv'))

In [20]:
# Display the first five rows of the grouped dataframe

df_group.head()

,start_station_name,end_station_name,total_trips
0,1 Ave & E 110 St,1 Ave & E 110 St,791
1,1 Ave & E 110 St,1 Ave & E 18 St,2
2,1 Ave & E 110 St,1 Ave & E 30 St,4
3,1 Ave & E 110 St,1 Ave & E 39 St,1
4,1 Ave & E 110 St,1 Ave & E 44 St,12


In [21]:
# Check the first 5 rows and the columns

df.head(5)

,Unnamed: 0,ride_id,bicycle_type,rent_start_time,rent_end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_latitude,start_longitude,end_latitude,end_longitude,customer_type,trip_duration,date,avg_temp,trip_count,value
0,0,9D0DC440CB40CF8E,electric_bike,2022-08-27 13:56:47.728,2022-08-27 14:02:56.651,Flatbush Ave & Ocean Ave,3704.04,3 St & Prospect Park West,3865.05,40.663657,-73.963014,40.668132,-73.973638,casual,6,2022-08-27,27.8,1,1
1,1,2214991DFBE5C4D7,electric_bike,2022-08-20 10:37:02.756,2022-08-20 10:45:56.631,Forsyth St\t& Grand St,5382.07,E 11 St & 1 Ave,5746.14,40.717798,-73.993161,40.729538,-73.984267,casual,9,2022-08-20,27.9,1,1
2,2,20C5D469563B6337,classic_bike,2022-08-31 18:55:03.051,2022-08-31 19:03:37.344,Perry St & Bleecker St,5922.07,Grand St & Greene St,5500.02,40.735354,-74.004831,40.721700,-74.002381,member,9,2022-08-31,25.6,1,1
3,3,3E8791885BC189D1,classic_bike,2022-08-02 08:05:00.250,2022-08-02 08:16:52.063,FDR Drive & E 35 St,6230.04,Grand Army Plaza & Central Park S,6839.1,40.744219,-73.971212,40.764397,-73.973715,member,12,2022-08-02,26.4,1,1
4,4,8DBCBF98885106CB,electric_bike,2022-08-25 15:44:48.386,2022-08-25 15:55:39.691,E 40 St & 5 Ave,6474.11,Ave A & E 14 St,5779.11,40.752052,-73.982115,40.730311,-73.980472,member,11,2022-08-25,28.1,1,1


In [22]:
# Drop Unnecessary Columns

df_stations = df.drop(columns=['Unnamed: 0', 'ride_id', 'bicycle_type', 'rent_start_time', 'rent_end_time', 'start_station_id', 'end_station_id', 'customer_type', 'trip_duration', 'date', 'avg_temp', 'value'])

In [23]:
# Check the first 5 rows and the columns

df_stations.head(5)

,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude,trip_count
0,Flatbush Ave & Ocean Ave,3 St & Prospect Park West,40.663657,-73.963014,40.668132,-73.973638,1
1,Forsyth St\t& Grand St,E 11 St & 1 Ave,40.717798,-73.993161,40.729538,-73.984267,1
2,Perry St & Bleecker St,Grand St & Greene St,40.735354,-74.004831,40.721700,-74.002381,1
3,FDR Drive & E 35 St,Grand Army Plaza & Central Park S,40.744219,-73.971212,40.764397,-73.973715,1
4,E 40 St & 5 Ave,Ave A & E 14 St,40.752052,-73.982115,40.730311,-73.980472,1


In [24]:
import gc

# 1) Build a UNIQUE lookup for start stations (one row per start_station_name)
start_lookup = (
    df_stations[['start_station_name', 'start_latitude', 'start_longitude']]
    .dropna(subset=['start_station_name'])
    .drop_duplicates(subset=['start_station_name'])
    .rename(columns={'start_latitude': 'start_lat', 'start_longitude': 'start_lon'})
)

# 2) Build a UNIQUE lookup for end stations (one row per end_station_name)
end_lookup = (
    df_stations[['end_station_name', 'end_latitude', 'end_longitude']]
    .dropna(subset=['end_station_name'])
    .drop_duplicates(subset=['end_station_name'])
    .rename(columns={'end_latitude': 'end_lat', 'end_longitude': 'end_lon'})
)

gc.collect()

0

In [25]:
# Merge start coords
df_final = df_group.merge(start_lookup, how='left', on='start_station_name')

# Merge end coords
df_final = df_final.merge(end_lookup, how='left', on='end_station_name')

df_final.head()

,start_station_name,end_station_name,total_trips,start_lat,start_lon,end_lat,end_lon
0,1 Ave & E 110 St,1 Ave & E 110 St,791,40.792337,-73.93824,40.792327,-73.938300
1,1 Ave & E 110 St,1 Ave & E 18 St,2,40.792337,-73.93824,40.733812,-73.980544
2,1 Ave & E 110 St,1 Ave & E 30 St,4,40.792337,-73.93824,40.741444,-73.975361
3,1 Ave & E 110 St,1 Ave & E 39 St,1,40.792337,-73.93824,40.747140,-73.971130
4,1 Ave & E 110 St,1 Ave & E 44 St,12,40.792337,-73.93824,40.750020,-73.969053


In [26]:
# Count missing latitude and longitude values for start and end station coordinates

df_final[['start_lat','start_lon','end_lat','end_lon']].isna().sum()

start_lat    0
start_lon    0
end_lat      0
end_lon      0
dtype: int64

In [27]:
# Count how many station pairs exceed different total trip thresholds and display the results

count_100 = (df_final['total_trips'] > 100).sum()
count_250 = (df_final['total_trips'] > 250).sum()
count_500 = (df_final['total_trips'] > 500).sum()
count_750 = (df_final['total_trips'] > 750).sum()

count_100, count_250, count_500, count_750

(70335, 23036, 6916, 2925)

In [29]:
# Create a dataframe of station pairs with more than 750 total trips and reset the index

df_750 = df_final[df_final['total_trips'] > 750].reset_index(drop=True)

In [30]:
# Display the number of rows and columns in the df_750 dataframe

df_750.shape

(2925, 7)

In [31]:
# Display the minimum total_trips value in the df_750 dataframe

df_750['total_trips'].min()

751

In [32]:
# Display the maximum total_trips value in the df_750 dataframe

df_750['total_trips'].max()

11999

In [33]:
# Check the first 5 rows and the columns

df_750.head(5)

,start_station_name,end_station_name,total_trips,start_lat,start_lon,end_lat,end_lon
0,1 Ave & E 110 St,1 Ave & E 110 St,791,40.792337,-73.938240,40.792327,-73.938300
1,1 Ave & E 110 St,2 Ave & E 96 St,1354,40.792337,-73.938240,40.783964,-73.947167
2,1 Ave & E 110 St,Lenox Ave & W 111 St,792,40.792337,-73.938240,40.798786,-73.952300
3,1 Ave & E 110 St,Lexington Ave & E 111 St,1259,40.792337,-73.938240,40.795412,-73.944123
4,1 Ave & E 16 St,1 Ave & E 16 St,1214,40.732219,-73.981656,40.732219,-73.981656


In [34]:
# Export the reduced dataframe with trips over 750 to a CSV file

df_750.to_csv('df_reduced_map_data_750.csv', index=False)

In [35]:
# Display the number of rows and columns in the merged dataframe

df_750.shape

(2925, 7)

In [36]:
# Create a Kepler.gl interactive map using the final dataframe and set the map height to 750

m = KeplerGl(height = 750, width = 1000, data={"data_1": df_750})
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1':                 start_station_name              end_station_name  total_trips  \
0   …

In [84]:
# Store the current Kepler.gl map configuration in a variable

config = m.config

In [85]:
# Display the current Kepler.gl map configuration stored in the config variable

config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'p2su76',
     'type': 'point',
     'config': {'dataId': 'data_1',
      'label': 'start',
      'color': [38, 71, 172],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'start_lat', 'lng': 'start_lon'},
      'isVisible': True,
      'visConfig': {'radius': 10,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'thickness': 2,
       'strokeColor': None,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radiusRange': [0, 

In [86]:
# Save the Kepler.gl map configuration to a JSON file named config.json

import json
with open("config.json", "w") as outfile:
    json.dump(config, outfile)

In [87]:
# Save the interactive Kepler.gl map to an HTML file with the specified configuration

m.save_to_html(file_name = 'NY_CitiBike_Trips_Map_750.html', read_only = False, config = config)

Map saved to NY_CitiBike_Trips_Map_750.html!


In [37]:
# Export "df_final" DataFrame to Csv

df_final.to_csv(os.path.join(path, 'CB_Data', 'Prepared Data', 'df_final.csv'))